In [83]:
import os, os.path as op
import shutil
import yaml
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import pandas as pd
import xarray as xr
from wand.image import Image as WImage

## Defining the home and data directories
tutorial_home_dir = os.path.abspath(os.getcwd())
work_dir = os.path.abspath(os.getcwd())
print("Tutorial directory: ", tutorial_home_dir)
print("Work directory: ", work_dir)

# Verifying if RAiDER is installed correctly
try:
    import RAiDER
except:
    raise Exception('RAiDER is missing from your PYTHONPATH')

os.chdir(work_dir)

Tutorial directory:  /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial
Work directory:  /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial


### Helpers

In [2]:
def new_yaml_group(example_yaml, dct_group, dst='tmp.yaml'):
    """ Write a temporary yaml file with the new 'value' for 'key', preserving parms in example_yaml"""
    with open(example_yaml, 'r') as f:
        try:
            params = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)
            raise ValueError(f'Something is wrong with the yaml file {example_yaml}')
    
    params = {**params, **dct_group}
    dst = op.join(os.path.dirname(example_yaml), dst)
#     print (params)
    
    with open(dst, 'w') as fh:
        yaml.dump(params, fh, default_flow_style=False)
    
    print ('Wrote new cfg file:', dst)
    return dst

In [3]:
yaml_base = os.path.join(tutorial_home_dir, 'example_yamls', 'raider_example_LA_bbox.yaml')

In [4]:
# check the base file works
!raider.py {yaml_base}

Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.12, 29.00, -113.12, 39.00
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 29.00, -113.00, 39.00
Starting to run the weather model cube calculation
Time: 2020-01-03 00:00:00
Max integration height is 15000.0 m
Output cube projection is GEOGCRS["Hu Tzu Shan 1950",DATUM["Hu Tzu Shan 1950",ELLIPSOID["International 1924",6378388,297,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]],AXIS["geodetic longitude (Lon)",east,ORDER[2],ANGLEUNIT["degree",0.0174532925199433]],USAGE[SCOPE["Geodesy."],AREA[

## Models

In [68]:
from RAiDER.models.allowed import ALLOWED_MODELS
models = ALLOWED_MODELS.copy()
models.remove('NCMR')
print ('Testing Models:', models)

Testing Models ['ERA5', 'ERA5T', 'HRRR', 'GMAO', 'HRES']


In [84]:
## may want to test downloading
shutil.rmtree('weather_files')

In [85]:
for model in models:
    grp  = {'weather_model': model}
    cfg  = new_yaml_group(yaml_base, grp)
    print ('Running:', model)
    !raider.py {cfg} 
    

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/example_yamls/tmp.yaml
Running: ERA5
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Weather model ERA-5 is available from 1950-01-01 00:00:00-Present
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-11-29 18:30:52,889 INFO Welcome to the CDS
2022-11-29 18:30:52,890 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-complete
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: Insecure

## Date Options

In [5]:
# date start only is default

In [6]:
# date start and end
grp = {'date_group': {'date_start' : 20200103, 'date_end': 20200107}}
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/example_yamls/tmp.yaml
Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.12, 29.00, -113.12, 39.00
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 29.00, -113.00, 39.00
Starting to run the weather model cube calculation
Time: 2020-01-03 00:00:00
Max integration height is 15000.0 m
Output cube projection is GEOGCRS["Hu Tzu Shan 1950",DATUM["Hu Tzu Shan 1950",ELLIPSOID["International 1924",6378388,297,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]]

Output height range is 0.0 to 1000.0
Invalid extension ('/Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/GMAO_tropo_20200107T000000_ztd', '.GTiff') for cube. Defaulting to .nc
Finished writing data to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/GMAO_tropo_20200107T000000_ztd.nc


In [7]:
# date start and end every 2 days
grp = {'date_group': {'date_start' : 20200103, 'date_end': 20200107, 'date_step': 2}}
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/example_yamls/tmp.yaml
Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.12, 29.00, -113.12, 39.00
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 29.00, -113.00, 39.00
Starting to run the weather model cube calculation
Time: 2020-01-03 00:00:00
Max integration height is 15000.0 m
Output cube projection is GEOGCRS["Hu Tzu Shan 1950",DATUM["Hu Tzu Shan 1950",ELLIPSOID["International 1924",6378388,297,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]]

In [27]:
# list of dates
grp = {'date_group': {'date_list': [20200110, 20200115]}}
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/example_yamls/tmp.yaml
Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200110
Max integration height is 15000.0 m
Beginning weather model pre-processing
Weather model GMAO is available from 2014-02-20 00:00:00-Present
Number of weather model nodes: 247950
Shape of weather model: (45, 38, 145)
Bounds of the weather model: 28.50/39.50/-123.75/-112.19 (SNWE)
Weather model: GMAO
Mean value of the wet refractivity: 8.561081
Mean value of the hydrostatic refractivity: 104.999130

======Weather Model class object=====
Weather model time: 2020-01-10 00:00:00
Latitude resolution: 0.25
Longitude resolution: 0.3125
Native projection: epsg:4326
ZMIN: -100.0
ZMAX: 15000.0
k1 = 0.776
k2 = 0.233
k3 = 3750.0
Humidity type = q
Class name: gmao
Datas

## Time Options

In [28]:
# start time only is default

In [29]:
# start and end time
grp = {'time_group': {'time': '00:00:00', 'end_time': '06:00:00'}}
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/example_yamls/tmp.yaml
Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.12, 29.00, -113.12, 39.00
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 29.00, -113.00, 39.00
Starting to run the weather model cube calculation
Time: 2020-01-03 00:00:00
Max integration height is 15000.0 m
Output cube projection is GEOGCRS["Hu Tzu Shan 1950",DATUM["Hu Tzu Shan 1950",ELLIPSOID["International 1924",6378388,297,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]]

## AOI Options

In [30]:
# bbox 
!raider.py {yaml_base}

Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.12, 29.00, -113.12, 39.00
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 29.00, -113.00, 39.00
Starting to run the weather model cube calculation
Time: 2020-01-03 00:00:00
Max integration height is 15000.0 m
Output cube projection is GEOGCRS["Hu Tzu Shan 1950",DATUM["Hu Tzu Shan 1950",ELLIPSOID["International 1924",6378388,297,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]],AXIS["geodetic longitude (Lon)",east,ORDER[2],ANGLEUNIT["degree",0.0174532925199433]],USAGE[SCOPE["Geodesy."],AREA[

In [55]:
# bbox global for GNSS
grp  = {'aoi_group': {'bounding_box': '-90 90 -180 180'}}
cfg  = new_yaml_group(yaml_base, grp)

!raider.py {cfg} --dostep download_gnss

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/example_yamls/tmp.yaml
Run routine processing with raider.py on steps: ['download_gnss']
--------------------------------------------------
11489 stations were found in [-90.0, 90.0, 180.0, 360.0]
8931 stations were found in [-90.0, 90.0, 0.0, 180.0]


In [56]:
# geocoded file (GUNW)
aria = op.join(op.expanduser('~'), 'data', 'VLM', 'Sentinel1', 'LA_ARIA', 'products')
prod = 'S1-GUNW-D-R-071-tops-20200130_20200124-135156-34956N_32979N-PP-913f-v2_0_4.nc'
grp  = {'aoi_group': {'geocoded_file': op.join(aria, prod)}}
cfg  = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/example_yamls/tmp.yaml
Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.12, 29.00, -113.12, 39.00
Extent of the input is (xmin, ymin, xmax, ymax): -119.36, 32.98, -116.24, 34.96
Starting to run the weather model cube calculation
Time: 2020-01-03 00:00:00
Max integration height is 15000.0 m
Output cube projection is GEOGCRS["Hu Tzu Shan 1950",DATUM["Hu Tzu Shan 1950",ELLIPSOID["International 1924",6378388,297,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]]

In [57]:
# geocoded file (SRTM DEM)
aria = op.join(op.expanduser('~'), 'data', 'VLM', 'Sentinel1', 'LA_ARIA', 'DEM')
prod = 'SRTM_3arcsec.dem'
grp  = {'aoi_group': {'geocoded_file': op.join(aria, prod)}}
cfg  = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/example_yamls/tmp.yaml
Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.12, 29.00, -113.12, 39.00
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 29.00, -113.00, 39.00
Starting to run the weather model cube calculation
Time: 2020-01-03 00:00:00
Max integration height is 15000.0 m
Output cube projection is GEOGCRS["Hu Tzu Shan 1950",DATUM["Hu Tzu Shan 1950",ELLIPSOID["International 1924",6378388,297,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]]

In [58]:
# ISCE lat/lon files
path_test = os.path.join(op.expanduser('~'), 'Google_Drive', 'BZ_RAiDER', 'LOS_SV_experiments', 'geom')
lats = op.join(path_test, 'lat1.geo')
lons = op.join(path_test, 'lon1.geo')
grp = {'aoi_group': {'lat_file':lats, 'lon_file':lons}, 
       # hampton roads
      'date_group': {'date_start': '20181113'},
      'time_group': {'time': '23:00:00'}

      }
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/example_yamls/tmp.yaml
Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20181113
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-76.88, 36.50, -75.94, 37.25
Extent of the input is (xmin, ymin, xmax, ymax): -76.65, 36.67, -75.93, 37.31
Starting to run the weather model cube calculation
Time: 2018-11-13 23:00:00
Max integration height is 15000.0 m
Output cube projection is GEOGCRS["Hu 

In [59]:
# gnss station file
yaml_gnss = yaml_base.replace('bbox', 'GNSS')
!raider.py {yaml_gnss}

Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.12, 29.00, -113.12, 39.00
Extent of the input is (xmin, ymin, xmax, ymax): -122.96, 29.03, -113.01, 39.00
Beginning DEM calculation
Finished writing data to file


## Height Options

In [60]:
# list of heights
!raider.py {yaml_base}

Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.12, 29.00, -113.12, 39.00
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 29.00, -113.00, 39.00
Starting to run the weather model cube calculation
Time: 2020-01-03 00:00:00
Max integration height is 15000.0 m
Output cube projection is GEOGCRS["Hu Tzu Shan 1950",DATUM["Hu Tzu Shan 1950",ELLIPSOID["International 1924",6378388,297,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]],AXIS["geodetic longitude (Lon)",east,ORDER[2],ANGLEUNIT["degree",0.0174532925199433]],USAGE[SCOPE["Geodesy."],AREA[

In [61]:
# dem
# geocoded file (SRTM DEM)
aria = op.join(op.expanduser('~'), 'data', 'VLM', 'Sentinel1', 'LA_ARIA', 'DEM')
prod = 'SRTM_3arcsec.dem'
grp  = {'height_group': {'dem': op.join(aria, prod), 'use_dem_latlon': True}}
        
cfg  = new_yaml_group(yaml_base, grp)

!raider.py {cfg}

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_tutorial/example_yamls/tmp.yaml
Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.12, 29.00, -113.12, 39.00
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 29.00, -113.00, 39.00
Beginning DEM calculation
/Users/buzzanga/Miniconda3/envs/RAiDER/lib/python3.10/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
Finished writing data to file


In [64]:
# geocoded hgt file from ISCE
!raider.py {yaml_base}

Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.12, 29.00, -113.12, 39.00
Extent of the input is (xmin, ymin, xmax, ymax): -123.00, 29.00, -113.00, 39.00
Starting to run the weather model cube calculation
Time: 2020-01-03 00:00:00
Max integration height is 15000.0 m
Output cube projection is GEOGCRS["Hu Tzu Shan 1950",DATUM["Hu Tzu Shan 1950",ELLIPSOID["International 1924",6378388,297,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]],AXIS["geodetic longitude (Lon)",east,ORDER[2],ANGLEUNIT["degree",0.0174532925199433]],USAGE[SCOPE["Geodesy."],AREA[

In [65]:
# gnss station file
yaml_gnss = yaml_base.replace('bbox', 'GNSS')
!raider.py {yaml_gnss}

Run routine processing with raider.py on steps: ['calculate_delays']
--------------------------------------------------
Starting to run the weather model calculation
Time type: <class 'datetime.datetime'>
Time: 20200103
Max integration height is 15000.0 m
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-123.12, 29.00, -113.12, 39.00
Extent of the input is (xmin, ymin, xmax, ymax): -122.96, 29.03, -113.01, 39.00
Beginning DEM calculation
Finished writing data to file


## LOS Options?

## Projection Options

In [41]:
# os.chdir('UTM_DEM')

In [42]:
## UTM DEM with WGS Model
# !raider.py raiderDelay_utm_CA-ERA5.yaml

In [43]:
## UTM DEM with UTM Model
# !raider.py raiderDelay_utm_CA-HRRR.yaml

In [44]:
## WGS DEM with UTM Model
# !raider.py raiderDelay_geo_CA-HRRR.yaml

In [45]:
# os.chdir('..')